# Multi-Agent Consensus Voting System
## Proof of Concept for Vedic Astrology Platform

This notebook demonstrates the consensus voting mechanism for reconciling interpretations from multiple specialized LLM agents.

**Key Features:**
- Weighted voting by domain type (career vs marriage have different weights)
- Confidence-adjusted contributions
- Conflict detection (threshold-based)
- Multiple resolution strategies
- Clean JSON output ready for API

---
*Author: Randhy Paul | December 2025*

## 1. Data Structures

In [ ]:
from dataclasses import dataclass
from typing import List, Dict
from enum import Enum
import json

class ResolutionStrategy(Enum):
    UNANIMOUS = "unanimous"
    WEIGHTED_MAJORITY = "weighted_majority"
    NUANCE_ARBITRATION = "nuance_arbitration"
    MATH_OVERRIDE = "math_override"

@dataclass
class AgentResponse:
    """Structured response from each specialized agent."""
    agent_id: str
    domain: str
    interpretation: str
    score: float  # 0-100
    confidence: float  # 0-1
    supporting_factors: List[str]
    contradicting_factors: List[str]

    def to_dict(self) -> dict:
        return {
            "agent_id": self.agent_id,
            "domain": self.domain,
            "score": self.score,
            "confidence": self.confidence
        }

@dataclass
class ConsensusResult:
    """Final consensus output."""
    domain: str
    final_score: float
    final_interpretation: str
    confidence: float
    agreement_level: str
    strategy_used: ResolutionStrategy
    conflicts_detected: int
    conflicts_resolved: int
    agent_contributions: Dict[str, float]

    def to_dict(self) -> dict:
        return {
            "domain": self.domain,
            "final_score": round(self.final_score, 2),
            "confidence": round(self.confidence, 3),
            "agreement_level": self.agreement_level,
            "strategy_used": self.strategy_used.value,
            "conflicts_detected": self.conflicts_detected,
            "conflicts_resolved": self.conflicts_resolved
        }

print("Data structures loaded successfully!")

## 2. Consensus Engine

In [ ]:
class ConsensusEngine:
    """
    Multi-agent consensus with weighted voting and conflict resolution.
    """

    # Agent weights vary by domain
    DOMAIN_WEIGHTS = {
        "career": {
            "integration_specialist": 0.30,
            "mathematics_validator": 0.15,
            "risk_assessor": 0.25,
            "nuance_specialist": 0.30
        },
        "marriage": {
            "integration_specialist": 0.25,
            "mathematics_validator": 0.10,
            "risk_assessor": 0.25,
            "nuance_specialist": 0.40  # Higher for marriage domain
        }
    }

    CONFLICT_THRESHOLD = 15.0

    def __init__(self):
        self.resolution_log = []

    def calculate_consensus(self, responses: List[AgentResponse], domain: str) -> ConsensusResult:
        weights = self.DOMAIN_WEIGHTS.get(domain, self.DOMAIN_WEIGHTS["career"])

        # Step 1: Calculate weighted scores
        weighted_sum = 0.0
        weight_total = 0.0

        for response in responses:
            agent_weight = weights.get(response.agent_id, 0.25)
            effective_weight = agent_weight * response.confidence
            weighted_sum += response.score * effective_weight
            weight_total += effective_weight

        initial_score = weighted_sum / weight_total if weight_total > 0 else 50.0

        # Step 2: Detect conflicts
        conflicts = self._detect_conflicts(responses, initial_score)

        # Step 3: Resolve conflicts
        if conflicts:
            final_score, strategy = self._resolve_conflicts(responses, conflicts, initial_score, domain)
        else:
            final_score = initial_score
            strategy = ResolutionStrategy.UNANIMOUS

        # Step 4: Calculate agreement level
        score_range = max(r.score for r in responses) - min(r.score for r in responses)
        agreement_level = "high" if score_range <= 10 else ("medium" if score_range <= 20 else "low")

        # Step 5: Final confidence
        avg_confidence = sum(r.confidence for r in responses) / len(responses)
        agreement_bonus = 0.1 if agreement_level == "high" else (-0.1 if agreement_level == "low" else 0)
        final_confidence = min(1.0, avg_confidence + agreement_bonus)

        return ConsensusResult(
            domain=domain,
            final_score=final_score,
            final_interpretation=self._generate_interpretation(final_score, agreement_level, domain),
            confidence=final_confidence,
            agreement_level=agreement_level,
            strategy_used=strategy,
            conflicts_detected=len(conflicts),
            conflicts_resolved=len(conflicts),
            agent_contributions={r.agent_id: weights.get(r.agent_id, 0.25) * r.confidence for r in responses}
        )

    def _detect_conflicts(self, responses, mean_score):
        return [r for r in responses if abs(r.score - mean_score) > self.CONFLICT_THRESHOLD]

    def _resolve_conflicts(self, responses, conflicts, initial_score, domain):
        # Check for nuance arbitration in marriage domain
        nuance_conflict = next((c for c in conflicts if c.agent_id == "nuance_specialist"), None)
        if nuance_conflict and domain == "marriage":
            self.resolution_log.append({"strategy": "NUANCE_ARBITRATION", "reason": "Marriage domain prioritizes nuance"})
            return 0.5 * nuance_conflict.score + 0.5 * initial_score, ResolutionStrategy.NUANCE_ARBITRATION

        # Default: weighted majority using top 3 by confidence
        sorted_responses = sorted(responses, key=lambda r: r.confidence, reverse=True)[:3]
        recalc_sum = sum(r.score * r.confidence for r in sorted_responses)
        recalc_weight = sum(r.confidence for r in sorted_responses)
        self.resolution_log.append({"strategy": "WEIGHTED_MAJORITY", "reason": "Using top 3 agents by confidence"})
        return recalc_sum / recalc_weight, ResolutionStrategy.WEIGHTED_MAJORITY

    def _generate_interpretation(self, score, agreement, domain):
        tier = "excellent" if score >= 80 else ("good" if score >= 65 else ("moderate" if score >= 50 else "challenging"))
        return f"{domain.title()} shows {tier} potential (score: {score:.1f}/100). Agreement: {agreement}."

print("Consensus Engine loaded!")

## 3. Test Cases

In [ ]:
def get_career_responses():
    """Simulated responses for career domain - HIGH AGREEMENT expected"""
    return [
        AgentResponse("integration_specialist", "career", "Strong 10th house", 78.5, 0.87, ["Jupiter aspects 10th"], []),
        AgentResponse("mathematics_validator", "career", "SAV 32/48", 72.0, 0.95, ["SAV above average"], []),
        AgentResponse("risk_assessor", "career", "Low risk", 81.0, 0.82, ["No Kemadruma"], []),
        AgentResponse("nuance_specialist", "career", "Steady rise", 75.0, 0.79, ["Neecha Bhanga"], [])
    ]

def get_marriage_responses():
    """Simulated responses for marriage domain - CONFLICT expected (nuance diverges)"""
    return [
        AgentResponse("integration_specialist", "marriage", "Venus strong", 70.0, 0.85, ["Venus in own sign"], []),
        AgentResponse("mathematics_validator", "marriage", "SAV 28/48", 58.0, 0.92, ["Average SAV"], []),
        AgentResponse("risk_assessor", "marriage", "Manglik cancelled", 62.0, 0.88, ["Cancellation applies"], []),
        AgentResponse("nuance_specialist", "marriage", "D9 excellent", 88.0, 0.75, ["D9 Venus exalted"], [])  # DIVERGENT!
    ]

print("Test cases ready!")

## 4. Run Tests

In [ ]:
engine = ConsensusEngine()

print("="*60)
print("TEST 1: CAREER DOMAIN (Expected: High Agreement)")
print("="*60)

career_responses = get_career_responses()
print("\nAgent Responses:")
for r in career_responses:
    print(f"  {r.agent_id}: score={r.score}, confidence={r.confidence}")

result1 = engine.calculate_consensus(career_responses, "career")
print("\nConsensus Result:")
print(json.dumps(result1.to_dict(), indent=2))

In [ ]:
print("="*60)
print("TEST 2: MARRIAGE DOMAIN (Expected: Conflict + Resolution)")
print("="*60)

marriage_responses = get_marriage_responses()
print("\nAgent Responses:")
for r in marriage_responses:
    print(f"  {r.agent_id}: score={r.score}, confidence={r.confidence}")

result2 = engine.calculate_consensus(marriage_responses, "marriage")
print("\nConsensus Result:")
print(json.dumps(result2.to_dict(), indent=2))

if engine.resolution_log:
    print("\nResolution Log:")
    for log in engine.resolution_log:
        print(f"  Strategy: {log['strategy']}")
        print(f"  Reason: {log['reason']}")

## 5. Summary

This prototype demonstrates:

| Feature | Status |
|---------|--------|
| Weighted voting by domain | OK |
| Confidence-adjusted contributions | OK |
| Conflict detection | OK |
| Resolution strategies | OK |
| Agreement level calculation | OK |
| JSON output for API | OK |

**Ready for production implementation with real LLM agents.**